In [1]:
import pandas as pd
import numpy as np

In [2]:
def change_features(df):
    df['AVG_returns'] = (df['AVG_returns'] + 1) ** 252 - 1
    df['Risk'] = (df['Risk'] + 1) ** np.sqrt(252) - 1
    df['Alpha'] = df['Alpha'].round(4)
    df['Drawdown'] = -df['Drawdown']
    return df 

In [3]:
dict_category2emb = {
    'topological' : [
        'PersImage_2',
        'PersLandscape_1'
        'PersImage_1',
        'PersLandscape_2',
        'topo_features'                 
      ],
    
    'dim_reduction' : ['umap', 'pca', 'fastica', 'tnse'],
    
    'transformers' : [ 'transformer_embds1',
 'transformer_embds2',
 'transformer_embds3',
 'transformer_embds_sum_23',
 'transformer_embds_sum_all',],
    
    'autoencoders' : [ 'autoencoder_conv',
 'autoencoder_lstm',
 'autoencoder_mlp',],
    
    'baselines' : ['sp500', 'sectors'],
    
    'interpretable' : [ 'tsfresh', 'table_finance_features'],
    
    'time_clustering' : ['KShape',
 'MSTcorr',
 'TimeSeriesKMeans'],
    
    'time_embedding' : ['ts2vec', 'signal2vec_embds']
}

dict_emb2cat = {method:categ for categ, list_methods in dict_category2emb.items() for method in list_methods }

# Data loading

In [4]:
fin_results_2012 = pd.read_csv('results/usa_2012/financial_metric_False.csv', index_col=0)
fin_results_2015 = pd.read_csv('results/usa_2015/financial_metric_False.csv', index_col=0)
fin_results_2018 = pd.read_csv('results/usa_2018/financial_metric_False.csv', index_col=0)

clust_results_2012 = pd.read_csv('results/usa_2012/clust_metric_False.csv', index_col=0)
clust_results_2015 = pd.read_csv('results/usa_2012/clust_metric_False.csv', index_col=0)
clust_results_2018 = pd.read_csv('results/usa_2012/clust_metric_False.csv', index_col=0)
fin_results_2012.head()

,AVG_returns,Risk,Beta,Alpha,Sharpe,VaR,Drawdown,Recovery,clust_model,emb_model
port,0.000517,0.016028,0.329642,0.000345,0.024955,-0.026447,-37.243770,385.0,KMeans,ts2vec
port,0.001062,0.014736,0.384230,0.000881,0.064101,-0.024314,-27.391429,259.0,Agg,ts2vec
port,0.000656,0.007743,0.752471,0.000414,0.069636,-0.012776,-11.716343,185.0,KMeans,PersImage_2
port,0.000659,0.008137,0.736411,0.000420,0.066596,-0.013426,-11.172857,183.0,Agg,PersImage_2
port,0.000633,0.015816,0.339472,0.000459,0.032596,-0.026096,-38.037391,385.0,KMeans,PersLandscape_1


In [5]:
def load_fin_clust(exp_name):
    dict_data = {
        "finance":pd.read_csv(f'results/{exp_name}/financial_metric_False.csv', index_col=0),
        "clustering":pd.read_csv(f'results/{exp_name}/clust_metric_False.csv', index_col=0)
    }
    return dict_data


In [6]:
experiments = ['usa_2012', 'usa_2015', 'usa_2018']
dict_results = {exp:load_fin_clust(exp) for exp in experiments}

In [7]:
def build_entire_table(fin_results, clust_results, dict_emb2cat, sort_metric='Risk'):
    fin_results = change_features(fin_results)

    df_to_rebuild = fin_results
    df_to_rebuild = pd.merge(clust_results.drop(columns=['clust_model']), df_to_rebuild, on='emb_model', how='outer')

    df_best_clustering = df_to_rebuild.sort_values([sort_metric], ascending=False)
    df_best_clustering = df_best_clustering.groupby('emb_model', as_index=False, sort=True).first()

    df_best_clustering['category'] = df_best_clustering['emb_model'].map(dict_emb2cat)
    return df_best_clustering.sort_values(sort_metric)

In [8]:
dict_results_selected = {exp:build_entire_table(
        dict_results[exp]['finance'], 
        dict_results[exp]['clustering'], 
        dict_emb2cat
        ) 
    for exp in experiments
}

In [9]:
all_experiments = pd.DataFrame([])

for exp_name, exp_data in dict_results_selected.items():
    exp_data['exp_name'] = exp_name
    
    all_experiments = pd.concat([all_experiments, exp_data])
    
print(all_experiments.shape)
all_experiments.head()

(63, 16)


,emb_model,DB,HC,Sil,hom,AVG_returns,Risk,Beta,Alpha,Sharpe,VaR,Drawdown,Recovery,clust_model,category,exp_name
1,PersImage_2,1.724300,61.553752,0.145029,0.099462,0.180643,0.137287,0.736411,0.0004,0.066596,-0.013426,11.172857,183.0,Agg,topological,usa_2012
10,sp500,NaN,NaN,NaN,NaN,0.073929,0.142009,1.000000,0.0000,0.019734,-0.013860,14.183248,253.0,sp500,baselines,usa_2012
0,PersImage_1,2.113002,38.591573,0.112135,0.085452,0.230194,0.143126,0.790175,0.0006,0.083330,-0.013962,11.145127,128.0,Agg,NaN,usa_2012
11,table_finance_features,1.243502,76.237093,0.159563,0.134707,0.293567,0.185312,0.500845,0.0008,0.084022,-0.017765,14.913612,184.0,KMeans,interpretable,usa_2012
9,sectors,4.019210,7.379893,-0.029870,1.000000,0.304948,0.204896,0.517192,0.0009,0.079542,-0.019488,25.386463,256.0,sectors,baselines,usa_2012


In [10]:
all_exp_pivot = all_experiments.pivot(index='emb_model', columns='exp_name', values=['Risk', 'Sharpe'])
all_exp_pivot.sort_values([('Risk', 'usa_2018')])

Risk                        Sharpe            \
exp_name                usa_2012  usa_2015  usa_2018  usa_2012  usa_2015   
emb_model                                                                  
sp500                   0.142009  0.136343  0.289158  0.019734  0.043638   
PersImage_2             0.137287  0.142336  0.335643  0.066596  0.128478   
PersImage_1             0.143126  0.147628  0.337164  0.083330  0.059114   
fastica                 0.279785  0.225570  0.361338  0.044366  0.078007   
topo_features           0.301772  0.179310  0.367651  0.041673  0.099889   
pca                     0.279785  0.225570  0.382380  0.044366  0.078007   
ts2vec                  0.287141  0.290068  0.390234  0.024955  0.098511   
transformer_embds2      0.211621  0.169785  0.390733  0.076999  0.147098   
table_finance_features  0.185312  0.176772  0.392020  0.084022  0.132535   
transformer_embds5      0.211621  0.160075  0.393402  0.076999  0.158944   
autoencoder_lstm        0.237643  0.219016  0.399237  0.065529  0.121365   
tsfresh                 0.267469  0.243374  0.403770  0.018736  0.113466   
transformer_embds3      0.214303  0.169452  0.407732  0.081541  0.148898   
PersLandscape_2         0.281690  0.218190  0.410371  0.041332  0.150288   
sectors                 0.204896  0.166120  0.415533  0.079542  0.135522   
transformer_embds4      0.236646  0.254810  0.419380  0.042340  0.133101   
transformer_embds1      0.280571  0.236600  0.419430  0.046580  0.122942   
umap                    0.220492  0.192902  0.425462  0.075281  0.110317   
PersLandscape_1         0.282873  0.278914  0.426972  0.032596  0.125858   
autoencoder_conv        0.316566  0.205836  0.441784  0.043027  0.143847   
autoencoder_mlp         0.297101  0.240412  0.450260  0.040065  0.113466   

                                  
exp_name                usa_2018  
emb_model                         
sp500                   0.014858  
PersImage_2             0.063872  
PersImage_1             0.050032  
fastica                 0.037374  
topo_features           0.049890  
pca                     0.025618  
ts2vec                  0.072554  
transformer_embds2      0.068911  
table_finance_features  0.051444  
transformer_embds5      0.066072  
autoencoder_lstm        0.067996  
tsfresh                 0.061094  
transformer_embds3      0.059415  
PersLandscape_2         0.066810  
sectors                 0.042644  
transformer_embds4      0.022160  
transformer_embds1      0.027812  
umap                    0.034040  
PersLandscape_1         0.068696  
autoencoder_conv        0.041836  
autoencoder_mlp         0.055189